# Hot Plasma II. Sandbox

Play around with some figures for Paper II

In [41]:
import sys
import os
from random import randint
from imp import reload
import pickle
import xml.etree.ElementTree as ET
import pandas as pd
from astroML import density_estimation
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import colorbar
from matplotlib.ticker import NullFormatter
import seaborn.apionly as sns
plt.style.use('papers')
sys.path.append(os.path.join(os.environ['RESEARCH_DIR'],'EBTEL_analysis/src/'))
import ebtel_dem as ebd
import ebtel_plot_em as ebpe
reload(ebd)
reload(ebpe)
%matplotlib inline

/opt/anaconda3/lib/python3.4/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [42]:
figname = 'ebtel_L40.0_tpulse200.0_alpha%s_%s_heating.lvl2_em_statistics.pickle'
data_root_ieq='/data/datadrive2/EBTEL_figs/%s_heating_runs/alpha%s'
data_root_nei='/data/datadrive2/IonPopSolver_figs/%s_heating_runs/alpha%s'

## Density at Onset of Heating Event for given $t_N$

At Peter's suggestion, try just comparing a $T,n$ timeseries for a single-heating frequency, one with the wait-time scaling and one without. 

In [ ]:
tn=5000
run_no=0
fnf_flag=False
fig,axes=plt.subplots(3,1,figsize=(10,8))
while not fnf_flag:
    #load data
    try:
        data_scaling=np.loadtxt(os.path.join(data_root%('electron','1.5'),'data',fn_temp%(tn,'-b1.0'),fn_temp%(tn,'-b1.0'))+'_%d.txt'%run_no)
        data_no_scaling=np.loadtxt(os.path.join(data_root%('electron','1.5'),'data',fn_temp%(tn,''),fn_temp%(tn,''))+'_%d.txt'%run_no)
    except FileNotFoundError:
        fnf_flag=True
        continue
    #cut
    ts,Tes,Tis,ns,qs=data_scaling[:,0],data_scaling[:,1],data_scaling[:,2],data_scaling[:,3],data_scaling[:,15]
    tns,Tens,Tins,nns,qns=data_no_scaling[:,0],data_no_scaling[:,1],data_no_scaling[:,2],data_no_scaling[:,3],data_no_scaling[:,15]
    #plotting
    axes[0].plot(ts,qs,color=sns.color_palette('deep')[0],linestyle='-',alpha=0.1)
    axes[0].plot(tns,qns,color=sns.color_palette('deep')[2],linestyle='-',alpha=0.1)
    axes[1].plot(ts,Tes,color=sns.color_palette('deep')[0],linestyle='-',alpha=0.1)
    #axes[1].plot(ts,Tis,color=sns.color_palette('deep')[2],linestyle='-',alpha=0.1)
    axes[1].plot(tns,Tens,color=sns.color_palette('deep')[2],linestyle='-',alpha=0.1)
    #axes[1].plot(tns,Tins,color=sns.color_palette('deep')[2],linestyle='--',alpha=0.1)
    axes[2].plot(ts,ns,color=sns.color_palette('deep')[0],linestyle='-',alpha=0.1)
    axes[2].plot(tns,nns,color=sns.color_palette('deep')[2],linestyle='-',alpha=0.1)
    #increment
    run_no+=1
    
plt.tight_layout()
plt.show()

In [ ]:
def get_onset_and_density(species,alpha,tn,scaling):
    "Read in heating onset times and associated densities"
    
    index_offset=1
    if species=='single':index_offset=0
    #get all data files
    data_files=[df for df in os.listdir(os.path.join(data_root%(species,str(alpha)),'data',fn_temp%(tn,scaling))) if 'heat_amp' not in df]
    res_dict={'amp':[],'density':[]}
    for df in data_files:
        #read in density
        data=np.loadtxt(os.path.join(data_root%(species,str(alpha)),'data',fn_temp%(tn,scaling),df))
        time,density=data[:,0],data[:,2+index_offset]
        #parse config file
        root=ET.parse(os.path.join(data_root%(species,str(alpha)),'config',fn_temp%(tn,scaling),df[:-3]+'xml')).getroot()
        #loop through configuration settings
        for child_ts,child_amp,i in zip(root.find('start_time_array'),root.find('amp_array'),range(int(root.find('num_events').text))):
            if i==0:
                continue
            else:
                #amplitude
                if alpha=='uniform':
                    res_dict['amp'].append(float(root.find('h_nano').text))
                else:
                    res_dict['amp'].append(float(child_amp.text))
                #density
                ts=float(child_ts.text)
                ind_ts=np.where(time>ts)[0]
                if len(ind_ts)<1:
                    print('start time out of range')
                    continue
                elif len(ind_ts)>len(time)-2:
                    print('indices too long for interpolation')
                    res_dict['density'].append(density[ind_ts[0]])
                else:
                    f=interpolate.interp1d(time[ind_ts[0]-2:ind_ts[0]+1],density[ind_ts[0]-2:ind_ts[0]+1])
                    res_dict['density'].append(f(ts))
                
    res_dict['amp']=np.array(res_dict['amp'])
    res_dict['density']=np.array(res_dict['density'])
                
    return res_dict

In [ ]:
dens_electron_alpha25=get_onset_and_density('electron',2.5,250,'-b1.0')

In [ ]:
#bins
_,kbins_amp = density_estimation.knuth_bin_width(dens_electron_alpha25['amp'],return_bins=True)
_,kbins_density = density_estimation.knuth_bin_width(dens_electron_alpha25['density'],return_bins=True)

In [ ]:
bincount,xedge,yedge=np.histogram2d(dens_electron_alpha25['amp'],dens_electron_alpha25['density'],
                                   bins=(kbins_amp,kbins_density),)
bincount_masked=np.ma.masked_where(bincount==0,bincount)
fig=plt.figure(figsize=(8,8))
ax=fig.gca()
X,Y=np.meshgrid(xedge,yedge)
im=ax.pcolormesh(X,Y,bincount_masked.T,cmap='Reds',
              norm=LogNorm(vmin=bincount_masked.min(), vmax=bincount_masked.max()))
ax.set_xlim([dens_electron_alpha25['amp'].min(),dens_electron_alpha25['amp'].max()])
ax.set_xscale('log')
ax.set_ylim([dens_electron_alpha25['density'].min(),dens_electron_alpha25['density'].max()])
ax.set_yscale('log')
fig.colorbar(im,ax=ax)
plt.show()

Now let's look at the case where the waiting time and event amplitude are independent.

In [ ]:
dens_electron_alpha25_no_scaling=get_onset_and_density('electron',2.5,250,'')

In [ ]:
#bins
_,kbins_amp_ns = density_estimation.knuth_bin_width(dens_electron_alpha25_no_scaling['amp'],return_bins=True)
_,kbins_density_ns = density_estimation.knuth_bin_width(dens_electron_alpha25_no_scaling['density'],return_bins=True)

In [ ]:
bincount,xedge,yedge=np.histogram2d(dens_electron_alpha25_no_scaling['amp'],dens_electron_alpha25_no_scaling['density'],
                                   bins=(kbins_amp_ns,kbins_density_ns),)
bincount_masked=np.ma.masked_where(bincount==0,bincount)
fig=plt.figure(figsize=(8,8))
ax=fig.gca()
X,Y=np.meshgrid(xedge,yedge)
im=ax.pcolormesh(X,Y,bincount_masked.T,cmap='Reds',
              norm=LogNorm(vmin=bincount_masked.min(), vmax=bincount_masked.max()))
ax.set_xlim([dens_electron_alpha25_no_scaling['amp'].min(),dens_electron_alpha25_no_scaling['amp'].max()])
ax.set_xscale('log')
#ax.set_ylim([dens_electron_alpha25_no_scaling['density'].min(),dens_electron_alpha25_no_scaling['density'].max()])
#ax.set_ylim([1.2e+8,1.6e+8])
ax.set_yscale('log')
fig.colorbar(im,ax=ax)
plt.show()

Next compare the cases where the event energies are or are not scaled.

In [ ]:
#setup figure
fig=plt.figure(figsize=(15,8))
p_spacing=0.01
left,bottom,width,height=0.2,0.1,0.65,0.9
ax=fig.add_axes((left,bottom,width,height))
#add an axes
axHisty = fig.add_axes((p_spacing,bottom,(1-10*p_spacing)*left,height)) # y histogram
#data set 1
bincount,xedge,yedge=np.histogram2d(dens_electron_alpha25['amp'],dens_electron_alpha25['density'],
                                   bins=(kbins_amp,kbins_density),)
bincount_masked=np.ma.masked_where(bincount==0,bincount)
X,Y=np.meshgrid(xedge,yedge)
#data set 2
bincount_ns,xedge_ns,yedge_ns=np.histogram2d(dens_electron_alpha25_no_scaling['amp'],dens_electron_alpha25_no_scaling['density'],
                                            bins=(kbins_amp_ns,kbins_density_ns))
bincount_masked_ns=np.ma.masked_where(bincount_ns==0,bincount_ns)
X_ns,Y_ns=np.meshgrid(xedge_ns,yedge_ns)
#plot data set 1
im=ax.pcolormesh(X,Y,bincount_masked.T,cmap='Reds',
              norm=LogNorm(vmin=bincount_masked.min(), vmax=bincount_masked.max()),alpha=0.75)
#plot data set 2
im_ns=ax.pcolormesh(X_ns,Y_ns,bincount_masked_ns.T,cmap='Blues',
              norm=LogNorm(vmin=bincount_masked_ns.min(), vmax=bincount_masked_ns.max()),linewidth=0,alpha=0.75)
im_ns.set_edgecolor('face')
#plotting 1D histogram
axHisty.hist(dens_electron_alpha25['density'],bins=kbins_density,orientation='horizontal',color=sns.color_palette('deep')[2],histtype='stepfilled',alpha=0.75)
axHisty.hist(dens_electron_alpha25_no_scaling['density'],bins=kbins_density_ns,orientation='horizontal',color=sns.color_palette('deep')[0],histtype='stepfilled',alpha=0.75)
#axes options
ax.set_xlim([min(dens_electron_alpha25_no_scaling['amp'].min(),dens_electron_alpha25['amp'].min()),max(dens_electron_alpha25_no_scaling['amp'].max(),dens_electron_alpha25['amp'].max())])
#ax.set_xlim([1e-3,10])
ax.set_xscale('log')
ax.set_ylim([min(dens_electron_alpha25_no_scaling['density'].min(),dens_electron_alpha25['density'].min()),max(dens_electron_alpha25_no_scaling['density'].max(),dens_electron_alpha25['density'].max())])
#ax.set_ylim([1e+8,1e+10])
ax.set_yscale('log')
axHisty.set_xlim(axHisty.get_xlim()[::-1])
axHisty.set_ylim(ax.get_ylim())
axHisty.set_yscale(ax.get_yscale())
#styling and labels
ax.yaxis.set_major_formatter(NullFormatter())
axHisty.set_ylabel(r'$n$ $(\mathrm{cm}^{-3})$')
ax.set_xlabel(r'$Q$ $(\mathrm{erg}$ $\mathrm{cm}^{-3}$ $\mathrm{s}^{-1})$')
#colorbar
cb_spacing=0.05
cb_width=0.02
cba1=fig.add_axes((left+width+p_spacing,0.1,cb_width,height))
cba2=fig.add_axes((left+width+p_spacing+cb_spacing+cb_width,0.1,cb_width,height))
fig.colorbar(im,cax=cba1)
fig.colorbar(im_ns,cax=cba2)
plt.show()

### Animating $(Q,n)$ distributions for varying $t_N$

Let's load all $(Q,n)$ for $250<t_N<5000$ s and then plot the histogram each time and save it.

In [ ]:
qn_scaling = []
qn_no_scaling = []
species='electron'
alpha=2.5
twait=np.arange(250,5250,250)

In [ ]:
for tw in twait:
    qn_scaling.append(get_onset_and_density(species,alpha,tw,'-b1.0'))
    qn_no_scaling.append(get_onset_and_density(species,alpha,tw,''))

In [ ]:
#save for later
with open('test_figs/histo2d.pickle','wb') as f:
    pickle.dump([qn_scaling,qn_no_scaling],f)

Once we've loaded the data (probably will take a while), we want to make many plots so that we can animate it.

In [ ]:
#axes sizing
p_spacing=0.01
left,bottom,width,height=0.2,0.1,0.65,0.9
cb_spacing=0.05
cb_width=0.02
for qns,qnns,tw in zip(qn_scaling,qn_no_scaling,twait):
    #set up the figure
    fig=plt.figure(figsize=(15,8))
    #add the axes
    ax=fig.add_axes((left,bottom,width,height))
    axHisty = fig.add_axes((p_spacing,bottom,(1-10*p_spacing)*left,height)) # y histogram
    #binning
    #scaling
    _,kba_s = density_estimation.knuth_bin_width(qns['amp'],return_bins=True)
    _,kbn_s = density_estimation.knuth_bin_width(qns['density'],return_bins=True)
    bc_s,x_s,y_s=np.histogram2d(qns['amp'],qns['density'],bins=(kba_s,kbn_s),)
    bc_s_masked=np.ma.masked_where(bc_s==0,bc_s)
    X_s,Y_s=np.meshgrid(x_s,y_s)
    #no scaling
    _,kba_ns = density_estimation.knuth_bin_width(qnns['amp'],return_bins=True)
    _,kbn_ns = density_estimation.knuth_bin_width(qnns['density'],return_bins=True)
    bc_ns,x_ns,y_ns=np.histogram2d(qnns['amp'],qnns['density'],bins=(kba_ns,kbn_ns),)
    bc_ns_masked=np.ma.masked_where(bc_ns==0,bc_ns)
    X_ns,Y_ns=np.meshgrid(x_ns,y_ns)
    #plotting
    #2D
    im_s=ax.pcolormesh(X_s,Y_s,bc_s_masked.T,cmap='Reds',norm=LogNorm(vmin=1, vmax=500),alpha=0.65,linewidth=0)
    im_s.set_edgecolor('face')
    im_ns=ax.pcolormesh(X_ns,Y_ns,bc_ns_masked.T,cmap='Blues',norm=LogNorm(vmin=1, vmax=500),linewidth=0,alpha=0.65)
    im_ns.set_edgecolor('face')
    #1D
    axHisty.hist(qns['density'],bins=kbn_s,orientation='horizontal',color=sns.color_palette('deep')[2],histtype='stepfilled',alpha=0.75)
    axHisty.hist(qnns['density'],bins=kbn_ns,orientation='horizontal',color=sns.color_palette('deep')[0],histtype='stepfilled',alpha=0.75)
    #axes options
    #2D
    ax.set_xlim([8e-3,2])
    ax.set_xscale('log')
    ax.set_ylim([9e+7,8e+9])
    ax.set_yscale('log')
    #1D
    axHisty.set_xlim([650,0])
    axHisty.set_ylim(ax.get_ylim())
    axHisty.set_yscale(ax.get_yscale())
    #styling and labels
    ax.yaxis.set_major_formatter(NullFormatter())
    axHisty.set_ylabel(r'$n$ $(\mathrm{cm}^{-3})$')
    ax.set_xlabel(r'$Q$ $(\mathrm{erg}$ $\mathrm{cm}^{-3}$ $\mathrm{s}^{-1})$')
    ax.set_title(r'$t_N=%d$ s'%tw)
    #add the colorbar
    cba1=fig.add_axes((left+width+p_spacing,0.1,cb_width,height))
    cba2=fig.add_axes((left+width+p_spacing+cb_spacing+cb_width,0.1,cb_width,height))
    fig.colorbar(im,cax=cba1)
    fig.colorbar(im_ns,cax=cba2)
    #print 
    plt.savefig('test_figs/hist2d_tn%d.pdf'%tw,format='pdf',dpi=1000)